In [ ]:
import seamless
from seamless import context, cell, reactor, transformer
ctx = context()
from seamless.websocketserver import websocketserver
websocketserver.start()
print("""If you are seeing this page via local port forwarding to an ssh server, please forward port {0} also
e.g.: 
    ssh -L {0}:localhost:{0} ssh-server

If you choose another port on localhost, please inform the websocketserver:
    ssh -L 7777:localhost:{0} ssh-server
    
    websocketserver.public_port = 7777
""".format(websocketserver.port))

In [ ]:
from seamless.lib.ngl import ngl
ctx.ngl = ngl(["pdb"], remote_ngl = True)
ctx.pdb = cell("text")
ctx.representations = cell("cson")
ctx.ngl_html = cell(("text", "html"))
ctx.ngl.html.connect(ctx.ngl_html)
ctx.pdb.connect(ctx.ngl.data_pdb)
ctx.representations.connect(ctx.ngl.representations)

In [ ]:
ctx.pdb.set(open("1AVX-bound.pdb").read())
ctx.representations.set("""
[
  {
    color: "red"
    sele: ":A"
  }
  {
    color: "blue"
    sele: ":B"
  }
  {
    repr: "ball+stick"
    color: "green"
    sele: "560-570:B"
  }
]
""")


In [ ]:
ctx.status()

In [ ]:
# External browser (Qt) (doesn't work remotely)
from seamless.lib.gui.browser import browse
browse(ctx.ngl_html)

In [ ]:
# External browser (Chrome) (doesn't work remotely)
from seamless.lib import link
link(ctx.ngl_html, ".", "test-ngl.html")
!google-chrome test-ngl.html

In [ ]:
# Embedded browsing
# Execute the "hack" cell as well (three cells below) to make it work properly (Jupyter/browser bug)
import IPython
h = IPython.display.HTML(ctx.ngl_html.value)
IPython.display.display_html(h,  metadata={"isolated": True})

## Representations
See http://arose.github.io/ngl/api/manual/usage/molecular-representations.html

In [ ]:
import ipywidgets
from seamless.gui.widgetlink import widgetlink
t = ipywidgets.Textarea()
obs = widgetlink(ctx.representations, t, as_data=True)
t.layout.height = "400px"
t.layout.width = "800px"
t

In [ ]:
%%javascript
// Hack to make the rendering work
// This is a Jupyter/browser bug: https://github.com/jupyter/notebook/issues/1282
v = document.getElementsByTagName("iframe");
for (var i = 0; i < v.length; i++) {
        elem = v[i];
        if (elem.className == "box-flex1")
            elem.style.height = "480px";
}    

In [ ]:
# For template development
from seamless.lib import link
link(ctx.ngl.tmpl.cell())